In [5]:
import pygis
import ee 
import geemap
ee.Initialize()

In [6]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NASA SRTM
dem = ee.Image('USGS/SRTMGL1_003')
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': 'terrain',
}
Map.addLayer(dem, dem_vis, 'SRTM DEM')

# Add 5-year Landsat TOA composite
landsat = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')
landsat_vis = {'bands': ['B4', 'B3', 'B2'], 'gamma': 1.4}
Map.addLayer(landsat, landsat_vis, "Landsat", False)

# Add US Census States
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [9]:
out_dem_stats= 'dem_stats.csv'
geemap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000, return_fc=False)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\dem_stats.csv


In [11]:
geemap.csv_to_df(out_dem_stats).sort_values('mean')

,mean,STATENS,GEOID,AWATER,LSAD,STUSPS,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
30,13.904218,1779781,10,1399985648,0,DE,10,A,38.998566,5,3,Delaware,-75.441644,G4000,5045925646,0000000000000000000b
1,14.699425,1779809,69,4644252461,0,MP,69,A,14.936784,0,9,Commonwealth of the Northern Mariana Islands,145.601021,G4000,472292529,00000000000000000023
0,15.149698,1802710,78,1550236201,0,VI,78,A,18.326748,0,9,United States Virgin Islands,-64.971251,G4000,348021896,00000000000000000022
27,21.218275,294478,12,31361101223,0,FL,12,A,28.457430,5,3,Florida,-82.409148,G4000,138949136250,00000000000000000001
3,26.543959,1802701,60,1307243754,0,AS,60,A,-14.267159,0,9,American Samoa,-170.668267,G4000,197759063,00000000000000000029
39,30.862419,1629543,22,23753621895,0,LA,22,A,30.863437,7,3,Louisiana,-91.798717,G4000,111897594374,00000000000000000014
2,36.152070,1802705,66,934337453,0,GU,66,A,13.438289,0,9,Guam,144.772949,G4000,543555840,00000000000000000024
34,43.438403,1702382,11,18687198,0,DC,11,A,38.904103,5,3,District of Columbia,-77.017229,G4000,158340391,00000000000000000035
5,57.769871,1219835,44,1323670487,0,RI,44,A,41.597419,1,1,Rhode Island,-71.527272,G4000,2677779902,00000000000000000005
11,65.266327,1779795,34,3544860246,0,NJ,34,A,40.107274,2,1,New Jersey,-74.665201,G4000,19047825980,0000000000000000000e


In [12]:
# remain the data in the cloud
fc= geemap.zonal_statistics(dem, states, out_dem_stats, statistics_type='MEAN', scale=1000, return_fc=True)

Computing statistics ...


In [13]:
Map= geemap.Map(center=[40, -100], zoom=4)
Map.addLayer(fc, {}, 'mean elevation')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [15]:
fc.aggregate_stats('ALAND')

In [16]:
Map = geemap.Map(center=[40, -100], zoom=4)

# Add NLCD data
dataset = ee.Image('USGS/NLCD_RELEASES/2019_REL/NLCD/2019')
landcover = dataset.select('landcover')
Map.addLayer(landcover, {}, 'NLCD 2019')

# Add US census states
states = ee.FeatureCollection("TIGER/2018/States")
style = {'fillColor': '00000000'}
Map.addLayer(states.style(**style), {}, 'US States')

# Add NLCD legend
Map.add_legend(title='NLCD Land Cover', builtin_legend='NLCD')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [17]:
nlcd_stats = 'nlcd_stats.csv'
geemap.zonal_stats_by_group(
    landcover,
    states,
    nlcd_stats,
    statistics_type='SUM',
    denominator=1000000,
    decimal_places=2,
)

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\nlcd_stats.csv


In [20]:
geemap.csv_to_df(nlcd_stats).sort_values('Class_82', ascending=False)

,Class_81,Class_71,Class_82,Class_95,Class_41,Class_52,Class_31,Class_42,Class_21,Class_43,...,STATEFP,FUNCSTAT,INTPTLAT,DIVISION,REGION,NAME,INTPTLON,MTFCC,ALAND,system:index
25,17824.32,1826.60,107288.83,958.38,9602.26,49.71,92.82,74.36,1460.59,496.06,...,19,A,42.070024,4,2,Iowa,-93.493347,G4000,144661267977,00000000000000000036
23,19052.11,74411.68,100762.74,328.05,8024.12,1957.52,85.20,29.39,3268.34,296.38,...,20,A,38.498546,4,2,Kansas,-98.383430,G4000,211755344060,0000000000000000002f
21,11348.82,41523.96,100580.32,5389.96,1900.72,11292.20,102.65,118.90,2465.34,63.14,...,38,A,47.442174,4,2,North Dakota,-100.460826,G4000,178707534813,00000000000000000020
14,10168.59,384.63,97470.21,421.21,19444.94,61.19,123.23,136.47,2193.84,884.25,...,17,A,40.102875,3,2,Illinois,-89.152611,G4000,143780567633,00000000000000000002
19,12869.78,1610.24,91131.41,16418.96,26124.37,1269.95,315.70,4193.42,2939.92,9654.79,...,27,A,46.315957,4,2,Minnesota,-94.199604,G4000,206228939448,00000000000000000003
40,74876.58,99457.22,88904.74,6837.04,15312.89,293157.57,1715.48,39426.19,8528.01,12327.64,...,48,A,31.434703,7,3,Texas,-99.281824,G4000,676653171537,00000000000000000019
20,1672.59,105503.75,80328.52,3452.45,1897.18,161.41,63.31,738.43,1686.54,168.63,...,31,A,41.543305,4,2,Nebraska,-99.811865,G4000,198956658395,00000000000000000011
24,15469.55,94803.71,67974.84,3399.34,631.40,2965.26,1308.42,5366.56,2152.66,123.99,...,46,A,44.446796,4,2,South Dakota,-100.238176,G4000,196346981786,00000000000000000032
49,7336.90,134905.75,57158.20,3144.34,536.43,91289.78,1572.94,77373.10,687.98,171.15,...,30,A,47.051177,8,4,Montana,-109.634817,G4000,376962738765,0000000000000000002e
18,6771.85,346.06,55227.55,149.93,20190.89,91.94,102.83,141.08,2195.64,854.94,...,18,A,39.901314,3,2,Indiana,-86.291913,G4000,92789302676,00000000000000000030
